In [2]:
modules = []

with open('example_input') as f:
    for line in f:
        line = line.strip().split()
        source = line[0]
        kind = "broadcaster"
        if source[0] == '%':
            kind = "flipflop"
            source = source[1:]
        if source[0] == '&':
            kind = "conjuction"
            source = source[1:]
        destinations = line[2:]
        modules.append([source, kind, [], destinations])

for item in modules:
    print(item)

['broadcaster', 'broadcaster', [], ['a,', 'b,', 'c']]
['a', 'flipflop', [], ['b']]
['b', 'flipflop', [], ['c']]
['c', 'flipflop', [], ['inv']]
['inv', 'conjuction', [], ['a']]
